In [25]:
import pyequion
import sympy
import scipy
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Create a system and save it to a C file

In [2]:
sys_eq = pyequion.create_equilibrium(['NaHCO3', 'CaCl2'])
pyequion.save_res_to_c_code(sys_eq, 'dummy', 'calc_cnv_res_equilibrium_NaHCO3_CaCl2', 
    # fixed_temperature=25.0
)


Check the file system for the created header and source files

## Generating Cython module for calling the C code

Reference: https://www.sympy.org/scipy-2017-codegen-tutorial/

1. Firstly a magic cell is used to create the build configuration file
1. Nextly, the cython file that will bridge the python interpreter with the c-function is defined and make it available to python interpreter

In [3]:
%%writefile calc_cnv_res_equilibrium_NaHCO3_CaCl2.pyxbld
import numpy

#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'calc_cnv_res_equilibrium_NaHCO3_CaCl2.c'],
                     include_dirs=['.', numpy.get_include()])

Writing calc_cnv_res_equilibrium_NaHCO3_CaCl2.pyxbld


In [4]:
%%cython_pyximport calc_cnv_res_equilibrium_NaHCO3_CaCl2
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API

# here we just replicate the function signature from the header
cdef extern from "calc_cnv_res_equilibrium_NaHCO3_CaCl2.h":
    void calc_cnv_res_equilibrium_NaHCO3_CaCl2(double T, double *concs, double *x, double *res)

# here is the "wrapper" signature that conforms to the odeint interface
def cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(double T, cnp.ndarray[cnp.double_t, ndim=1] concs, cnp.ndarray[cnp.double_t, ndim=1] x):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] res = np.empty(x.size, dtype=np.double)
    # now call the C function
    calc_cnv_res_equilibrium_NaHCO3_CaCl2(<double> T, <double *> concs.data, <double *> x.data, <double *> res.data)
    # return the result
    return res

In [40]:
def wrapper_py_res_func(x, T, concs):
    return cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(T, concs, x)

In [9]:
sol = pyequion.solve_solution({'NaHCO3': 10, 'CaCl2': 5})
cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(25.0+273.15, np.array([10.0, 5.0]), sol.x)

array([-5.68434189e-14,  4.44089210e-16,  0.00000000e+00,  2.84217094e-14,
        0.00000000e+00,  7.77156117e-16,  0.00000000e+00,  1.77635684e-15,
        0.00000000e+00, -2.27373675e-13,  9.99000000e+00,  9.99000000e+00,
        4.99500000e+00,  9.99000000e+00,  0.00000000e+00])

## Generating C Code for the Jacobian

In [11]:
pyequion.save_jacobian_of_res_to_c_code(sys_eq, 'dummy', 'calc_cnv_jac_equilibrium_NaHCO3_CaCl2', 
    # fixed_temperature=25.0
)


In [12]:
%%writefile calc_cnv_jac_equilibrium_NaHCO3_CaCl2.pyxbld
import numpy

#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'calc_cnv_jac_equilibrium_NaHCO3_CaCl2.c'],
                     include_dirs=['.', numpy.get_include()])

Writing calc_cnv_jac_equilibrium_NaHCO3_CaCl2.pyxbld


In [13]:
%%cython_pyximport calc_cnv_jac_equilibrium_NaHCO3_CaCl2
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API

# here we just replicate the function signature from the header
cdef extern from "calc_cnv_jac_equilibrium_NaHCO3_CaCl2.h":
    void calc_cnv_jac_equilibrium_NaHCO3_CaCl2(double T, double *x, double *res)

# here is the "wrapper" signature that conforms to the odeint interface
def cy_calc_cnv_jac_equilibrium_NaHCO3_CaCl2(double T, cnp.ndarray[cnp.double_t, ndim=1] x):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] J = np.empty((x.size*x.size), dtype=np.double)
    # now call the C function
    calc_cnv_jac_equilibrium_NaHCO3_CaCl2(<double> T, <double *> x.data, <double *> J.data)
    # return the result

    mat_J = J.reshape((x.size, -1))
    return mat_J

In [42]:
def wrapper_py_jac_func(x, T, concs):
    return cy_calc_cnv_jac_equilibrium_NaHCO3_CaCl2(T, x)

In [34]:
sol = pyequion.solve_solution({'NaHCO3': 10, 'CaCl2': 5})

In [24]:
J = cy_calc_cnv_jac_equilibrium_NaHCO3_CaCl2(25, np.full(15, -3.0))
J.shape
J

array([[ 0.        , -0.97797761, -0.97797761,  0.02202239,  0.        ,
         0.02202239,  0.08808954,  0.02202239,  0.        ,  0.        ,
         0.08808954,  0.02202239,  0.        ,  0.02202239,  0.02202239],
       [-1.        , -0.02361803, -0.02361803,  0.97638197,  0.        ,
         0.97638197, -0.09447213, -0.02361803,  0.        ,  0.        ,
        -0.09447213, -0.02361803,  0.        , -0.02361803, -0.02361803],
       [ 0.        , -0.0029818 , -1.0029818 ,  0.9970182 , -1.        ,
        -0.0029818 , -0.01192722, -0.0029818 ,  0.        ,  0.        ,
        -0.01192722, -0.0029818 ,  0.        , -0.0029818 , -0.0029818 ],
       [ 0.        , -0.04311272,  0.95688728, -0.04311272,  0.        ,
        -1.04311272,  0.8275491 , -0.04311272,  0.        ,  0.        ,
        -0.1724509 , -0.04311272,  0.        , -0.04311272, -0.04311272],
       [ 0.        , -0.0447669 , -0.0447669 ,  0.9552331 ,  0.        ,
        -0.0447669 ,  0.82093239, -1.0447669 , 

In [ ]:
## Solving the system



In [23]:
np.full(15, -3)

array([-3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3])

['NaHCO3',
 'OH-',
 'H+',
 'Na+',
 'NaOH',
 'HCO3-',
 'CO3--',
 'NaCO3-',
 'Na2CO3',
 'CO2',
 'Ca++',
 'CaOH+',
 'CaCO3',
 'CaHCO3+',
 'Cl-',
 'H2O']

In [43]:
concs = np.array([10e-3,5e-3])
root_sol = scipy.optimize.root(wrapper_py_res_func, sol.x, args=(25+273.15, concs), jac=wrapper_py_jac_func)
root_sol

    fjac: array([[ 0.00000000e+00,  9.99999992e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  8.87383607e-05,  8.87383607e-05,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00, -4.07451850e-14, -3.41143460e-07,
        -4.53142697e-06, -4.79142557e-06, -7.45642413e-06,
         2.36144688e-06, -2.45265355e-06, -9.71015202e-06,
        -4.83215964e-06,  2.29286940e-10,  2.29286940e-10,
         0.00000000e+00,  0.00000000e+00, -2.37473280e-06],
       [-2.04965197e-06,  2.00890275e-14, -4.99999326e-01,
         5.00001731e-01,  2.36539103e-06,  3.68102530e-06,
        -5.00000660e-01, -4.99998283e-01,  4.79362690e-06,
         2.38550029e-06, -1.13192465e-10, -1.13192465e-10,
         0.00000000e+00,  0.00000000e+00,  1.20238230e-06],
       [ 8.18558193e-06, -8.84827294e-07, -3.18104067e-01,
        -9.33160640e-02, -4.11758287e-01, -

In [52]:
wrapper_py_jac_func(np.full(15,-3.0), 25+273.15, concs)
# wrapper_py_res_func(np.full(15,-3.0), 25+273.15, concs)
wrapper_py_res_func(sol.x, 25+273.15, concs)

array([-5.68434189e-14,  4.44089210e-16,  0.00000000e+00,  2.84217094e-14,
        0.00000000e+00,  7.77156117e-16,  0.00000000e+00,  1.77635684e-15,
        0.00000000e+00, -2.27373675e-13, -1.73472348e-18, -1.73472348e-18,
       -2.60208521e-18,  0.00000000e+00,  0.00000000e+00])